# Extracting data for SRV training

In [1]:
import os
import sys
import time
import logging
import fiona
import rasterio
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
import shapely.speedups
from IPython.display import display
from omegaconf.listconfig import ListConfig
from multiprocessing import Pool, cpu_count

# import cuspatial
import folium
import osgeo.gdal
import tensorflow as tf
from pathlib import Path
from glob import glob
from shapely.geometry import box
from skimage.transform import resize
from sklearn.cluster import KMeans
from matplotlib import colors

sys.path.append('/explore/nobackup/people/jacaraba/development/tensorflow-caney')

from tensorflow_caney.config.cnn_config import Config
from tensorflow_caney.utils.system import seed_everything
from tensorflow_caney.utils.model import load_model

from tensorflow_caney.utils.data import modify_bands, \
    get_mean_std_metadata
from tensorflow_caney.utils import indices
from tensorflow_caney.inference import inference
from tensorflow_caney.utils.data import normalize_image, rescale_image, \
    standardize_batch, standardize_image

# from sklearn.model_selection import train_test_split
# from tensorflow.keras.optimizers import Adam
# from sklearn.metrics import mean_squared_error

/gpfsm/ccds01/home/appmgr/app/jupyterhub/ilab/kernel/lib/python3.7/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.10) doesn't match a supported version!
  RequestsDependencyWarning)
2022-10-07 14:11:41.555112: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


ImportError: cannot import name 'unet_regression' from 'tensorflow_caney.networks' (/explore/nobackup/people/jacaraba/development/tensorflow-caney/tensorflow_caney/networks/__init__.py)

In [ ]:
# model_filename = '/explore/nobackup/projects/3sl/development/cnn_landcover/crop.exp6/landcover-crop43-0.05.hdf5'
model_filename = '/explore/nobackup/projects/3sl/development/cnn_landcover/otcb.all/landcover-otcb59-0.26.hdf5'
model = load_model(
    model_filename=model_filename,
    #model_dir=os.path.join(conf.data_dir, 'model')
)

In [ ]:
#model.predict(np.random.randint(255, size=(1, 512, 512, 4)))

In [ ]:
srv_data = '/explore/nobackup/projects/3sl/data/VHR/SRV/M1BS/*-toa.tif'
srv_label_dir = '/explore/nobackup/projects/3sl/labels/landcover/srv'
output_dir = '/explore/nobackup/projects/3sl/development/cnn_landcover/crop.srv.v1/metadata'
general_crs = "EPSG:32628"
start_year = 2009
end_year = 2021 # 2020

In [ ]:
# create output directory
os.makedirs(output_dir, exist_ok=True)

In [ ]:
def get_worldview_scenes_geometry(data_regex: str, crs: str = None):
    """
    Get WorldView from local EVHR TIFs.
    Improvements:
        - this could be done way faster with multiprocessing
        - optional GPU or CPU versions, add GPU inside function
    """
    # get the paths/filenames of the world view imagery available
    filenames = []
    if isinstance(data_regex, list):
        for regex in data_regex:
            filenames.extend(glob(regex))
    else:
        filenames = glob(data_regex)

    # define variables to store the output of the searches
    scene_ids_list, bounds_ids_list, years_ids_list, \
        study_area_list = [], [], [], []

    if crs is None:
        crs = rasterio.open(filenames[0]).crs

    for f in filenames:
        study_area_list.append(f.split('/')[-3])
        scene_ids_list.append(f)
        years_ids_list.append(int(Path(f).stem[5:9]))
        bounds_ids_list.append(box(*rasterio.open(f).bounds))

    d = {
        'study_area': study_area_list,
        'scene_id': scene_ids_list,
        'acq_year': years_ids_list,
        'geometry': bounds_ids_list
    }
    return gpd.GeoDataFrame(d, crs=crs)

def filter_gdf_by_list(
            gdf,
            gdf_key: str = 'acq_year',
            isin_list: list = [],
            reset_index: bool = True
        ):
    """
    Filter GDF by year range.
    """
    return gdf[gdf[gdf_key].isin(isin_list)].reset_index(drop=True)

In [ ]:
# get evhr bounding boxes
evhr_gdf = get_worldview_scenes_geometry(srv_data, general_crs)
evhr_gdf = evhr_gdf.to_crs(4326)
display(evhr_gdf.head())

# create folium map
total_bounds = evhr_gdf.dissolve().to_crs('+proj=cea').centroid.to_crs(evhr_gdf.crs)
m = folium.Map(
    location=[total_bounds.y, total_bounds.x],
    zoom_start=8,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google'
)

for _, r in evhr_gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(str(r['acq_year'])).add_to(geo_j)
    geo_j.add_to(m)
display(m)

In [ ]:
# reproject to match worldview projection
evhr_gdf = evhr_gdf.to_crs(epsg=general_crs.split(':')[-1])

In [ ]:
for year in range(start_year, end_year):

    # get label filename
    label_filename = glob(os.path.join(srv_label_dir, f'*{year}*.gpkg'))
    
    # skip iteration if no labels available for this year
    if not label_filename:
        continue
    
    label_filename = label_filename[0]
    
    # iterate over labels
    fields_gdf = gpd.read_file(label_filename)
    fields_gdf = fields_gdf.to_crs(epsg=general_crs.split(':')[-1])
    # print(fields_gdf)
    
    # iterate over 
    filtered_evhr_gdf = filter_gdf_by_list(evhr_gdf, 'acq_year', [int(Path(label_filename).stem[-4:])])
    # print(filtered_evhr_gdf)

    intersection = gpd.sjoin(
        fields_gdf, filtered_evhr_gdf[['study_area', 'scene_id', 'acq_year', 'geometry']],
        how='left', op='intersects'
    )
    intersection.dropna(
        axis=0,
        how='any',
        thresh=None,
        subset=None,
        inplace=True
    )
    intersection['acq_year'] = intersection['acq_year'].astype(int)
    print(f"{label_filename}, {intersection.shape[0]} matches.")
    
    output_filename = os.path.join(output_dir, f'{Path(label_filename).stem}.gpkg')

    # save geopackage file within the output_dir
    intersection.to_file(output_filename, driver='GPKG', layer='Intersection')

#srv_label_filenames = sorted(glob(srv_label))
#print(f"{len(srv_label_filenames)} label filenames.")

In [ ]:
# get evhr bounding boxes
# evhr_gdf = get_worldview_scenes_geometry(srv_data, general_crs)
# evhr_gdf = evhr_gdf.to_crs(4326)
intersection = intersection.to_crs(4326)
display(intersection.head())

# create folium map
total_bounds = intersection.dissolve().to_crs('+proj=cea').centroid.to_crs(intersection.crs)
m = folium.Map(
    location=[total_bounds.y, total_bounds.x],
    zoom_start=8,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google'
)

for _, r in intersection.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(str(int(r['acq_year']))).add_to(geo_j)
    geo_j.add_to(m)
display(m)

## Second Part: Extracting the actual tiles

Once we have a dataframe with the intersection elements - one polygon of the field that is present in the actual worldview image, we proceed to extract the tiles. Here we will take a deeper look at how to extract them in the proper order and location.

In [ ]:
regex = '/explore/nobackup/projects/3sl/development/cnn_landcover/crop.srv.v1/metadata/*.gpkg'
data_output_dir = '/explore/nobackup/projects/3sl/development/cnn_landcover/crop.srv.v1'
tile_size = 256
filenames = glob(regex)
len(filenames)

In [ ]:
import struct, numpy, pylab
import matplotlib.pyplot as plt

def extractWindow(imageDataset, pixelX, pixelY, pixelWidth, pixelHeight):
    # Extract raw data
    if type(imageDataset) is np.ndarray:
        matrix = imageDataset[int(pixelX):int(pixelX+pixelWidth), int(pixelY):int(pixelY+pixelHeight)]
    else:
        matrix = imageDataset.ReadAsArray(pixelX, pixelY, pixelWidth, pixelHeight)
        print("AQUI ESTA EL ERROR")
    #print(pixelX, pixelY, pixelWidth, pixelHeight)
    #plt.imshow(matrix[7, :, :] / 10000.0)
    #plt.show()
    #pylab.show()
    # Return
    return matrix

def extractCenteredWindow(imageDataset, pixelX, pixelY, pixelWidth, pixelHeight):
    centeredPixelX = pixelX - pixelWidth / 2
    centeredPixelY = pixelY - pixelHeight / 2
    return extractWindow(imageDataset, centeredPixelX, centeredPixelY, pixelWidth, pixelHeight)

def convertGeoLocationToPixelLocation(geoLocation):
    xGeo, yGeo = geoLocation[0], geoLocation[1]
    if g2 == 0:
        xPixel = (xGeo - g0) / float(g1)
        yPixel = (yGeo - g3 - xPixel*g4) / float(g5)
    else:
        xPixel = (yGeo*g2 - xGeo*g5 + g0*g5 - g2*g3) / float(g2*g4 - g1*g5)
        yPixel = (xGeo - g0 - xPixel*g1) / float(g2)
    return int(round(xPixel)), int(round(yPixel))

def convertGeoDimensionsToPixelDimensions(geoWidth, geoHeight, g1, g5):
    return int(round(abs(float(geoWidth) / g1))), int(round(abs(float(geoHeight) / g5)))

In [ ]:
%%time

import cv2
from skimage.draw import polygon
from sklearn.cluster import spectral_clustering
from skimage.filters import threshold_otsu, threshold_mean, threshold_isodata, threshold_local
from skimage.filters import threshold_otsu, rank
from skimage.morphology import disk
from  scipy import ndimage
from skimage.feature import canny
from skimage.filters import sobel
from scipy import ndimage as ndi
from skimage.segmentation import chan_vese


data_output_dir = '/explore/nobackup/projects/3sl/development/cnn_landcover/crop.srv.v1'

images_output_dir = os.path.join(data_output_dir, 'images')
labels_output_dir = os.path.join(data_output_dir, 'labels')

os.makedirs(images_output_dir, exist_ok=True)
os.makedirs(labels_output_dir, exist_ok=True)

k_means_predictor = None

for filename in filenames[4:6]:
    
    # read gpkg
    dataset_gdf = gpd.read_file(filename)
    #print(dataset_gdf.shape, dataset_gdf.crs)
    display(dataset_gdf.head())
    
    for index, row in dataset_gdf.iterrows():
        
        try:
            #print(row["scene_id"])
            output_data_filename = os.path.join(images_output_dir, f'{Path(row["scene_id"]).stem}_{str(index+1)}.npy')
            output_label_filename = os.path.join(labels_output_dir, f'{Path(row["scene_id"]).stem}_{str(index+1)}.npy')

            #------------------------------------------------------------------------------
            # here we extract the data tile
            #------------------------------------------------------------------------------
            imageDataset = osgeo.gdal.Open(row['scene_id'])

            polygon_centroid = row['geometry'].centroid
            print("centroid", polygon_centroid.x, polygon_centroid.y, polygon_centroid)

            g0, g1, g2, g3, g4, g5 = imageDataset.GetGeoTransform()
            print(
                f"g0={g0}, g1={g1}, g2={g2}, g3={g3}, g4={g4}, g5={g5}, {imageDataset.RasterXSize}, {imageDataset.RasterYSize}")

            windowPixelX, windowPixelY = convertGeoLocationToPixelLocation(
                [row['geometry'].centroid.x, row['geometry'].centroid.y])
            print("Pixel location: ", windowPixelX, windowPixelY)

            windowPixelWidth, windowPixelHeight = tile_size, tile_size
            data_matrix = extractCenteredWindow(
                imageDataset, windowPixelX, windowPixelY, windowPixelWidth, windowPixelHeight)
            data_matrix = np.moveaxis(data_matrix, 0, -1)

            # if nodata is present, skip
            if data_matrix.min() < 0:
                continue
                
            #------------------------------------------------------------------------------
            # here we extract the label tile
            #------------------------------------------------------------------------------
            label_mask = numpy.full((imageDataset.RasterXSize,imageDataset.RasterYSize), False)
            polygon_pixel_coords = np.apply_along_axis(
                convertGeoLocationToPixelLocation, axis=1, arr=np.array(list(row['geometry'].exterior.coords)))
            rr, cc = polygon(
                polygon_pixel_coords[:,0], polygon_pixel_coords[:,1],
                (imageDataset.RasterXSize,imageDataset.RasterYSize)
            )
            label_mask[cc, rr] = True

            # there is a strange represenation of where X and Y are location within the polygon
            # for now just invert the axis and call it a day
            label_matrix = extractCenteredWindow(
                label_mask.astype(int), windowPixelY, windowPixelX, windowPixelWidth, windowPixelHeight)

            #print("occurrence", np.count_nonzero(label_matrix == 1), np.unique(label_matrix))

            if np.count_nonzero(label_matrix == 1) < 10000: #20000:  # 512x512 / 2
                continue
            
            # assert if a tile is smaller than the given size we want
            if label_matrix.shape[0] != windowPixelWidth or label_matrix.shape[1] != windowPixelHeight:
                continue

            np.save(output_data_filename, data_matrix)
            np.save(output_label_filename, label_matrix)
            
            # grab the number of bands in the image, naip images have four bands
            nbands = data_matrix.shape[-1]
            band_s = 6

            # create an empty array in which each column will hold a flattened band
            flat_data = np.empty((data_matrix.shape[0]*data_matrix.shape[1], nbands))

            # loop through each band in the image and add to the data array
            data_matrix_2 = standardize_image(data_matrix/10000.0, 'local')
            
            for i in range(nbands):
                band = data_matrix_2[:, :, i]
                flat_data[:, i-1] = band.flatten()
                        
            #if k_means_predictor is None:
            k_means_predictor = KMeans(n_clusters=2, n_init=1, random_state=24, algorithm="elkan").fit(flat_data)
            
            #print(k_means_predictor.labels_)
            kmeans_predicted = k_means_predictor.predict(flat_data).reshape((data_matrix.shape[0], data_matrix.shape[1]))

            #predicted = spectral_clustering(data_matrix[:, :, 6], n_clusters=2, eigen_solver="arpack")

            #plt.figure(figsize = (6,6))
            #plt.imshow(data_matrix[:, :, 6] / 10000.0)
            #plt.imshow(label_matrix, alpha=0.5)
            #plt.imshow(predicted, alpha=0.2, cmap=colors.ListedColormap(['k','r']))
            #plt.show()
            
            #img = cv2.cvtColor(data_matrix[:, :, 4],cv2.COLOR_BGR2RGB)
            thresh_otsu = threshold_otsu(data_matrix[:, :, band_s])
            thresh_mean = threshold_mean(data_matrix[:, :, band_s])
            thresh_isodata = threshold_isodata(data_matrix[:, :, band_s])
            thresh_local = threshold_local(data_matrix[:, :, band_s], block_size=201, offset=10)
            local_otsu = rank.otsu(data_matrix[:, :, band_s], disk(15))
            #predicted = data_matrix[:, :, 6] > thresh
            #print(binary.shape)
            
            #final_label = label_matrix + (data_matrix[:, :, band_s] > thresh_mean)
            #final_label[final_label > 1] = 1
            # PREDICT TILE
            
            #pred_image = standardize_image(data_matrix/10000.0, 'local')
            #pred_image = model.predict(np.expand_dims(pred_image, axis=0))
            #pred_image = np.squeeze(np.argmax(pred_image, axis=-1))
            #print(pred_image.shape, np.unique(pred_image))
            
            #pred_image[pred_image == 1] = 0
            #pred_image[pred_image == 2] = 1
            #pred_image[pred_image == 3] = 0
            #pred_image[pred_image == 4] = 0

            #final_label = pred_image
            #ndvi = (data_matrix[:, :, 6] - data_matrix[:, :, 5]) / (data_matrix[:, :, 6] + data_matrix[:, :, 5])
            #print(ndvi.shape, ndvi.min(), ndvi.max())
            #ndvi = canny(data_matrix[:, :, 6]/10000.0)
            #ndvi = sobel(data_matrix[:, :, 6]/10000.0)
            #ndvi = ndi.binary_fill_holes(ndvi)
            print("sobel", ndvi.min(), ndvi.max())
            ndvi = chan_vese(data_matrix[:, :, 6]/10000.0)
            #ndvi = ndvi
            # final_label = ndimage.binary_fill_holes((data_matrix[:, :, band_s] > thresh_mean)).astype(int)

            
            fig, (ax1, ax2, ax3, ax4, ax5, ax6, ax7) = plt.subplots(1, 7, figsize=(20, 10) )
            ax1.imshow(data_matrix[:, :, band_s] / 10000.0)
            ax1.imshow(label_matrix, alpha=0.5)
            ax2.imshow(label_matrix, cmap=colors.ListedColormap(['k','r']))
            ax3.imshow(data_matrix[:, :, band_s] > thresh_otsu, cmap=colors.ListedColormap(['k','r']))
            ax4.imshow(data_matrix[:, :, band_s] > thresh_mean, cmap=colors.ListedColormap(['k','r']))
            ax5.imshow(data_matrix[:, :, band_s] > thresh_isodata, cmap=colors.ListedColormap(['k','r']))
            ax6.imshow(kmeans_predicted, cmap=colors.ListedColormap(['k','r']))
            #ax7.imshow(final_label, cmap=colors.ListedColormap(['k','r']))
            ax7.imshow(ndvi)
            
            ax1.set_title('NIR1 + Alpha Original Label')
            ax2.set_title('Label Matrix')
            ax3.set_title('Otsu Threshold')
            ax4.set_title('Mean Threshold')
            ax5.set_title('Isodata Threshold')
            ax6.set_title('K-Means')
            ax7.set_title('Final Label Tile (Mean Shift + Label)')

            plt.show()
            
            #print(data_matrix.shape, data_matrix[:, :, 6])
            #labels = spectral_clustering(data_matrix[:, :, 6], n_clusters=2, eigen_solver="arpack")
            #print(labels, labels.shape)
            
            # from sklearn.cluster import MeanShift
            
        except (AttributeError, IndexError) as e:
            print(e)
            continue
